In [3]:
import sys
sys.path.append('/SEE-MTDA/see')
from sc.mesher.mesher import Mesher
import numpy as np
import os
import glob
import pickle
from pathlib import Path
from sc.datasets.shared_utils import *
import open3d as o3d
from easydict import EasyDict
import yaml
from PIL import Image
from tqdm import tqdm

def cfg_from_yaml_file(cfg_file):
    cfg = EasyDict()
    print("\n----- Cfgs -----")
    with open(cfg_file, 'r') as f:
        yaml_config = yaml.safe_load(f)
        print(yaml.dump(yaml_config))

    cfg.update(EasyDict(yaml_config))

    return cfg

cfg_file = '/SEE-MTDA/see/sc/cfgs/KIT-LSEG-EXT_R6D-E3_SURFACEKNN_min10pts.yaml'
cfg = cfg_from_yaml_file(cfg_file)
mesher = Mesher(cfg=cfg, cfg_path=cfg_file)


----- Cfgs -----
DATASET:
  CAMERA_CHANNELS:
  - image_2
  CLASSES:
  - Car
  DET2D_MASK: true
  DET2D_MODEL: yolox
  NAME: kitti
  POINT_CLOUD_RANGE:
  - -70
  - -70
  - -2
  - 70
  - 70
  - 4
DATA_PATH: /SEE-MTDA/data/kitti
MESHER:
  EXPORT_NAME: R6D_E3_surfaceknn_k30
  MIN_LIDAR_PTS_TO_MESH: 10
  NAME: ext_mesh
  REPLACE_OBJECT_WITH_MESH:
    POINT_DISTANCE_THRESH: 0.1

Loading masks...
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
KittiObjects initialised!

Mesher Initialised!


# Export points in GT box

This is to assess shape completion with perfect data

In [32]:
mesher.min_lidar_pts_to_mesh = 5
save_dir = Path(f'/SEE-MTDA/data/shapenet/VC/kitti_vc/test')
# mesher.data_obj.nsweeps = 1

car_count = 0
max_num_cars = 5000

for sample_idx in tqdm(range(mesher.data_obj.__len__()), total=mesher.data_obj.__len__()):
    pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx)

    for object_id in range(len(pcd_gtboxes['gt_boxes'])):    

        object_pcd = pcd_gtboxes['pcd'].crop(pcd_gtboxes['gt_boxes'][object_id])

        if len(object_pcd.points) < mesher.min_lidar_pts_to_mesh:
            continue

        box_pts = np.asarray(pcd_gtboxes['gt_boxes'][object_id].get_box_points())
        label = pcd_gtboxes['xyzlwhry_gt_boxes'][object_id][:7]

        # Save object
        # Save partial, complete, labels
        partial_dir = save_dir / 'partial'
        partial_dir.mkdir(exist_ok=True, parents=True)

        label_dir = save_dir / 'label' 
        label_dir.mkdir(exist_ok=True, parents=True)

        fname =  f'frame-{sample_idx}_car-{object_id:03d}'    
        o3d.io.write_point_cloud(str(partial_dir / f'{fname}.pcd'), object_pcd)
        with open(str(label_dir / f'{fname}.pkl'), 'wb') as f:
            label = {'bbox_pts': box_pts, 
                     'gtbox': label,
                     'pc_id': fname}
            pickle.dump(label, f)
        car_count += 1
        if car_count >= max_num_cars:
            break
            
    if car_count >= max_num_cars:
        break
        
file_glob = glob.glob(f'{str(label_dir)}/*')
file_list = [fname.split('/')[-1].split('.')[0] for fname in file_glob]
with open(f'{save_dir}/file_list.txt','w') as f:
    f.writelines('\n'.join(file_list))
        
#         if mesher.data_obj.dataset_name == 'nuscenes':
#             fname = f'nsweeps-{mesher.data_obj.nsweeps}_frame-{sample_idx}_object-{object_id}_car_pts-{len(object_pcd.points)}'
#         else:
#             fname = f'frame-{sample_idx}_object-{object_id}_car_pts-{len(object_pcd.points)}'

#         car_dir = Path(f'{mesher.data_obj.root_dir}/exported/cars')
#         car_dir.mkdir(parents=True, exist_ok=True)
#         o3d.io.write_point_cloud(str(car_dir / f'{fname}.pcd'), object_pcd)

#         box_dir = Path(f'{mesher.data_obj.root_dir}/exported/bboxes')    
#         box_dir.mkdir(parents=True, exist_ok=True)
#         np.savetxt(str(box_dir / f'{fname}.txt'), box_pts, delimiter=' ')

#         print(f"frame-{sample_idx}: Exported {object_id}/{len(pcd_gtboxes['gt_boxes'])} | car_pts-{len(object_pcd.points)}")


 19%|█▉        | 1413/7481 [02:45<11:48,  8.56it/s] 


In [14]:
root_dir = Path('/SEE-MTDA/data/nuscenes/custom_t4025-v3980')
ftrain = open(str(root_dir / 'custom_train_split.txt'), 'r').read()            
scene_names = ftrain.split('\n')
print('Scene names len = ', len(scene_names))

fval = open(str(root_dir / 'custom_val_split.txt'), 'r').read()
# scene_names = fval.split('\n')
scene_names.extend(fval.split('\n'))
print('Scene names len = ', len(scene_names))

Scene names len =  100
Scene names len =  199


In [56]:
import glob

train_paths = glob.glob('/SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/*')
train = [ids.split('/')[-1] for ids in train_paths]
val = glob.glob('/SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/test/complete/*')
val = [ids.split('/')[-1] for ids in val]
intersect = list(set(train).intersection(set(val)))
len(intersect)

273

In [57]:
import shutil

for i in intersect:
    intersect_idx = train.index(i)
    path = train_paths[intersect_idx]
    shutil.rmtree(path)
    print(f'Removed {path}')

Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/eb1bd7389854311c14f284ebe538e531
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/aec393b8c73d3a8ae4bee94a37836d67
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/a5b7cc1a5ab2828b28b0b7b876595fb8
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/2254e63f695a1cccbbae182929bb7dd
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/63316c4cff51de417fb21fb4ec0d8d1b
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/43be650d6fd7468fd9952f9e00a53f0e
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/19042f5a90290859441c11ab4641b257
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/f3205edd456cf36feabc798aaf5f039f
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/bbc2f2ffeec425a38eac574d099cd5d2
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/91be65c386bafdd21aee99ace10c2d7c
Removed /SEE-MTDA/dat

Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/3dab4ef100f906c2bda733a39f84326d
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/ae1c1141ce4bfde9d66a73de5847ea37
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/206e39a5f67f4a4a81d0c86cc18e6647
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/c8fd6ee4bdbfa5cfbda3e4f05af7d436
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/6b0238b41337365a3330c5ee7577e4de
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/100c3076c74ee1874eb766e5a46fceab
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/468be6b80d1c7dd655ac255cda83e91
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/40c05e511efd3fa96945ae93118ea9d6
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/397d2e2b3e0988a2d3901a534bc610d8
Removed /SEE-MTDA/data/shapenet/VC/WAYMO_nviews-30/train/partial/1a30678509a1fdfa7fb5267a071ae23a
Removed /SEE-MTDA/dat

In [8]:
pcd_gtboxes = mesher.get_pcd_gtboxes(8)
box_pts = np.asarray(pcd_gtboxes['gt_boxes'][0].get_box_points())
box_pts

array([[ 2.19680528,  2.3956037 , -1.53011145],
       [ 5.30030254,  1.50050327, -1.53011145],
       [ 2.64019868,  3.9329398 , -1.53011145],
       [ 2.19680528,  2.3956037 ,  0.03988855],
       [ 5.74369594,  3.03783937,  0.03988855],
       [ 2.64019868,  3.9329398 ,  0.03988855],
       [ 5.30030254,  1.50050327,  0.03988855],
       [ 5.74369594,  3.03783937, -1.53011145]])

# Export points in mask/bbox

This is to test out lidar segmentation on the points within a box

In [18]:
mesher.data_obj.mask_dir

PosixPath('/SEE-MTDA/data/kitti/training/masks/yolox')

In [26]:
htc_mask.dataset['annotations']

[{'id': '1000000',
  'image_id': 0,
  'category_id': 0,
  'iscrowd': 0,
  'area': 5103,
  'bbox': [713.2767333984375,
   150.7246856689453,
   95.412109375,
   158.2196502685547],
  'segmentation': [[765,
    151,
    764,
    152,
    761,
    152,
    755,
    158,
    755,
    159,
    754,
    160,
    754,
    164,
    753,
    165,
    753,
    169,
    752,
    170,
    752,
    171,
    751,
    172,
    751,
    174,
    750,
    175,
    750,
    176,
    749,
    177,
    749,
    178,
    747,
    180,
    747,
    182,
    746,
    183,
    746,
    184,
    745,
    185,
    745,
    186,
    744,
    187,
    744,
    191,
    743,
    192,
    743,
    195,
    742,
    196,
    742,
    197,
    741,
    198,
    741,
    204,
    740,
    205,
    740,
    215,
    739,
    216,
    739,
    224,
    740,
    225,
    740,
    230,
    741,
    231,
    741,
    235,
    742,
    236,
    742,
    239,
    741,
    240,
    741,
    244,
    740,
    245,
    740,
   

In [24]:
from pycocotools.coco import COCO
mask_json = '/SEE-MTDA/data/kitti/training/masks/htc/image_2.json'
htc_mask = COCO(mask_json)

loading annotations into memory...
Done (t=1.34s)
creating index...
index created!


In [39]:
htcinst[0]['segmentation']

[[437,
  180,
  436,
  181,
  430,
  181,
  429,
  182,
  426,
  182,
  425,
  183,
  424,
  183,
  423,
  184,
  422,
  184,
  418,
  188,
  417,
  188,
  413,
  192,
  412,
  192,
  408,
  196,
  407,
  196,
  401,
  202,
  400,
  202,
  397,
  205,
  396,
  205,
  390,
  211,
  389,
  211,
  386,
  214,
  385,
  214,
  381,
  218,
  380,
  218,
  379,
  219,
  379,
  220,
  374,
  225,
  374,
  226,
  373,
  227,
  373,
  229,
  372,
  230,
  372,
  231,
  371,
  232,
  371,
  233,
  370,
  234,
  370,
  246,
  371,
  247,
  371,
  249,
  372,
  250,
  372,
  252,
  373,
  253,
  373,
  255,
  374,
  256,
  374,
  258,
  376,
  260,
  377,
  260,
  378,
  261,
  379,
  261,
  380,
  262,
  391,
  262,
  392,
  261,
  394,
  261,
  395,
  260,
  398,
  260,
  399,
  259,
  417,
  259,
  418,
  260,
  423,
  260,
  424,
  261,
  427,
  261,
  428,
  262,
  434,
  262,
  435,
  263,
  441,
  263,
  442,
  264,
  448,
  264,
  449,
  265,
  453,
  265,
  454,
  266,
  457,
  266,
  458,

In [54]:
idx = 876
aid = htc_mask.getAnnIds(imgIds=idx, catIds=[2])
htcinst = htc_mask.loadAnns(aid)
seg_mask = htc_mask.annToMask(htcinst[0])
print(seg_mask.shape)
print(mesher.data_obj.get_image(idx, 'image_2').shape)

(376, 1241)
(376, 1241, 3)


In [4]:
idx = 21
camera_channel='image_2'
img = mesher.data_obj.get_image(idx, channel=camera_channel)
pc_velo = mesher.data_obj.get_pointcloud(idx, append_labels=False)
calib = mesher.data_obj.get_calibration(idx)
imgfov = mesher.data_obj.map_pointcloud_to_image(pc_velo[:,:3], calib, img, min_dist=1.0)
imgfov['img_shape'] = img.shape[:2] # H, W
instances = mesher.data_obj.get_camera_instances(idx, channel=camera_channel)
            


In [8]:
sample_idx = 28
i_clouds = mesher.data_obj.get_mask_instance_clouds(sample_idx, use_bbox=True)
# pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx, add_ground_lift=True)    
# inst_labels = mesher.data_obj.get_camera_instances(sample_idx, channel=camera_channel)

In [14]:
from PIL import Image
from tqdm import tqdm

camera_channel = 'image_2'

export_tag = 'bbox'
if export_tag == 'bbox':
    use_bbox = True
else:
    use_bbox = False

instance_dataset_id = 0
for sample_idx in tqdm(range(mesher.data_obj.__len__()), total=mesher.data_obj.__len__()):
    
    i_clouds = mesher.data_obj.get_mask_instance_clouds(sample_idx, use_bbox=use_bbox)
    if len(i_clouds) == 0:
        continue
    
    pcd_gtboxes = mesher.get_pcd_gtboxes(sample_idx, add_ground_lift=True)    
    inst_labels = mesher.data_obj.get_camera_instances(sample_idx, channel=camera_channel)
        
    for object_id, i_cloud in enumerate(i_clouds):
        mask_pcd = convert_to_o3dpcd(i_clouds[object_id])
        cropped_pcds, num_pts_in_gt_for_mask = [], []        
        
        if len(i_cloud) < 10:
            continue
            
        for i in range(len(pcd_gtboxes['gt_boxes'])):    
            cropped = mask_pcd.crop(pcd_gtboxes['gt_boxes'][i])
            num_pts_in_gt_for_mask.append(len(cropped.points))
            cropped_pcds.append(cropped)            
            
        if not num_pts_in_gt_for_mask or max(num_pts_in_gt_for_mask) == 0:
            labelled_pts = np.hstack([i_cloud[:,:3], np.zeros((len(i_cloud),1))])
            box_pts = np.zeros((8,3))
            num_points_in_gt = 0
        else:
            gtbox_idx = np.argmax(np.array(num_pts_in_gt_for_mask)) 
            
            cropped_np = np.array(cropped_pcds[gtbox_idx].points)
            obj_pts_labelled = np.hstack([cropped_np, 10*np.ones((len(cropped_np),1))])

            dist = mask_pcd.compute_point_cloud_distance(cropped_pcds[gtbox_idx])
            cropped_inds = np.where(np.asarray(dist) < 0.01)[0]
            mask_without_obj = np.asarray(mask_pcd.select_by_index(cropped_inds, invert=True).points)
            mask_pts_labelled = np.hstack([mask_without_obj, np.zeros((len(mask_without_obj),1))])

            labelled_pts = np.vstack([mask_pts_labelled, obj_pts_labelled])
            box_pts = np.asarray(pcd_gtboxes['gt_boxes'][gtbox_idx].get_box_points())
            num_points_in_gt = len(cropped_np)
               
        # Save pcl
        fname = f"{sample_idx:06}_{object_id:03}"
        car_dir = Path(f'{mesher.data_obj.root_dir}/exported/{mesher.data_obj.dataset_cfg.DET2D_MODEL}/lidar_seg_data_groundlift/points_in_{export_tag}/pointcloud')
        car_dir.mkdir(parents=True, exist_ok=True)
        car_fname = str(car_dir / f'{fname}.bin')
        points = np.float32(labelled_pts)
        points.tofile(car_fname)
        
        # Save points labels
        label_dir = Path(f'{mesher.data_obj.root_dir}/exported/{mesher.data_obj.dataset_cfg.DET2D_MODEL}/lidar_seg_data_groundlift/points_in_{export_tag}/labels')
        label_dir.mkdir(parents=True, exist_ok=True)
        label_fname = str(label_dir / f'{fname}.label')
        labels = np.uint32(labelled_pts[:,3])
        labels.tofile(label_fname)
        
        # Save gt bboxes
        box_dir = Path(f'{mesher.data_obj.root_dir}/exported/{mesher.data_obj.dataset_cfg.DET2D_MODEL}/lidar_seg_data_groundlift/points_in_{export_tag}/gt_bboxes')    
        box_dir.mkdir(parents=True, exist_ok=True)
        np.savetxt(str(box_dir / f'{fname}.txt'), box_pts, delimiter=' ')
        
#         # Save img - this takes a while. Not necessary for cylinder3d
#         img_dir = Path(f'{mesher.data_obj.root_dir}/exported/{mesher.data_obj.dataset_cfg.DET2D_MODEL}/lidar_seg_data_groundlift/points_in_{export_tag}/image')    
#         img_dir.mkdir(parents=True, exist_ok=True)
#         img = mesher.data_obj.get_image(sample_idx, channel=camera_channel)
#         imglabels = inst_labels[object_id]
#         ann_img = draw_lidar_on_image([], img, instances=[imglabels], display=False)
#         Image.fromarray(ann_img).save(str(img_dir / f'{fname}.png'))

        # Save original filepath of input pointcloud
        origpath_dir = Path(f'{mesher.data_obj.root_dir}/exported/{mesher.data_obj.dataset_cfg.DET2D_MODEL}/lidar_seg_data_groundlift/points_in_{export_tag}/fpath')
        origpath_dir.mkdir(parents=True, exist_ok=True)
        with open(str(origpath_dir / f'{fname}.txt'), 'w') as f:
              f.write(f'{sample_idx:06}')
        
#         print(f"car #{instance_dataset_id}: Exported {object_id+1}/{len(i_clouds)} | total pts-{len(labelled_pts)}, gt pts - {num_points_in_gt}")
        instance_dataset_id = instance_dataset_id + 1

100%|█| 7481/7481 [25:27<00:00,  4.90it


# Import lidarseg predictions

In [29]:
sample_idx = 21
icloud_paths = glob.glob(f'{mesher.data_obj.root_dir}/exported/lidar_seg_data_groundlift/points_in_bbox/icloud/*.pcd')
i_cloud = [np.asarray(o3d.io.read_point_cloud(path).points) for path in icloud_paths if  f'{sample_idx:06}_' in path]
list_mesh_instances = mesher.mesh_det_pts(i_cloud, sample_idx)
# frame_labels = [np.fromfile(paths, dtype=np.uint32).reshape((-1,1)) for paths in label_paths if f'{sample_idx:06}_' in paths]
# frame_frustums = [np.fromfile(paths, dtype=np.float32).reshape((-1,4)) for paths in frustum_paths if f'{sample_idx:06}_' in paths]
original_pcd = convert_to_o3dpcd(mesher.data_obj.get_pointcloud(sample_idx))
final_pcd = mesher.replace_pts_with_mesh_pts(original_pcd, list_mesh_instances, label_points=False, point_dist_thresh=mesher.mesher_cfg.REPLACE_OBJECT_WITH_MESH.POINT_DISTANCE_THRESH)        

In [7]:
o3d.io.write_point_cloud('temp.pcd', convert_to_o3dpcd(final_pcd), write_ascii=False)

True